In [1]:
import pandas as pd
import geonamescache as gc
import re
from unidecode import unidecode

In [2]:
# function 
def remove_duplicate(lists):
    lists = list(set(lists))
    to_remove_lists = [x for x in lists for y in lists if x!=y and x in y ] 
    output = [x for x in lists if x not in to_remove_lists]   
    return output

In [3]:
# varaiable prep 
# Get Geoname
geo = gc.GeonamesCache()
countries = geo.get_countries()
cities = geo.get_cities()
countries_list = [countries[x]['name'] for x in countries ]
cities_list = [cities[x]['name'] for x in cities ]

In [4]:
# Create Regex
def create_regex(data):
    unicode_data = unidecode(data)
    if data != unicode_data :
        regex = fr'\b({data}|{unicode_data})\b'
    else:
        regex = fr'\b{data}\b'
    return re.compile(regex)
    
def Regex_to_name(text, dictionary):
    match_name = ['']
    for regex, name in sorted(dictionary.items(),key=lambda x: x[1]):
        if regex.search(text) :
            match_name.append(name)
    return match_name   

In [5]:
# Build Dict and Create Regex
Country_Dict = {create_regex(country): country for country in countries_list}
City_Dict = {create_regex(city): city for city in cities_list}

In [6]:
# Dataframe
cc_names = pd.DataFrame(columns=['headline', 'country', 'city'])

In [7]:
# file open 
headline =open("../data/headlines.txt", "r")

In [8]:

for line in headline.readlines():
    line = line.strip()
    countries_matches = Regex_to_name(line, Country_Dict)
    city_matches = Regex_to_name(line, City_Dict)
    
    if countries_matches != [''] and countries_matches != None :
        countries_matches = remove_duplicate(countries_matches) 
    else :
        countries_matches = None
        
    if city_matches != [''] and city_matches != None :  
        city_matches = remove_duplicate(city_matches)
    else :
        city_matches = None
        
    cc_names = cc_names.append({'headline': line,'country': countries_matches ,'city':  city_matches},ignore_index=True)
    

In [9]:
cc_names 

,headline,country,city
0,Zika Outbreak Hits Miami,None,[Miami]
1,Could Zika Reach New York City?,None,[New York City]
2,First Case of Zika in Miami Beach,None,[Miami Beach]
3,"Mystery Virus Spreads in Recife, Brazil",[Brazil],[Recife]
4,Dallas man comes down with case of Zika,None,[Dallas]
5,Trinidad confirms first Zika case,None,[Trinidad]
6,Zika Concerns are Spreading in Houston,None,[Houston]
7,Geneve Scientists Battle to Find Cure,None,[Genève]
8,The CDC in Atlanta is Growing Worried,None,[Atlanta]
9,Zika Infested Monkeys in Sao Paulo,None,[São Paulo]


In [10]:
headline.close()